## Installing and importing modules

In [2]:
!pip install colorama
!pip install genanki
!pip install googletrans==3.1.0a0
!pip install gtts
!pip install multiprocessing
!pip install pandas
!pip install selenium
!pip install threading
!pip install xlsxwriter
!pip install openpyxl
!pip install zipfile
from colorama import Fore,Back,Style,init #use colors
import genanki #integration with anki
from googletrans import Translator #translate the phrases
from gtts import gTTS #generate audios
import os #get the file directories
from multiprocessing import cpu_count #know the exact number of cores
import pandas as pd #handle the excel tables
from selenium import webdriver # open color custom website
import time #count the amount of time used in creating cards
from threading import Thread #acelerate the code
import xlsxwriter #excel tables
from zipfile import ZipFile #for some reason google colab don't download folders, so i'll convert the audio folder into winrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=12d50

## **How to use it**
1. Import your excel table to the file menu
2. Run the "Installing and import modules"
3. Click in "Run CardMaker"
4. Use the code as explained on my [Github](https://github.com/viniciusdutra314/CardMaker)
5. The code will generate an "audios.zip", you need to open it and past these audios file into the collection.media (example of directory C:\Users\dutra\AppData\Roaming\Anki2\Dutra\collection.media)
6. You also need to open the "nomeofthetable.apkg" file which is an anki deck

In [12]:
translator = Translator() #start translator
init(strip=False) #colors in the terminal
t_o=time.time() # counting time
threads=round(cpu_count()/2) #using half of the threads to acelerate the code...you can change this if you want

chosen_language=input("Choose the language that your cards and the interface will be translated (ex:pt,fr,en): ") #secret command, you can change the number of cores typing the number of cores after the chosen language (Ex: es !4)
    
if "!" in chosen_language:
  position=chosen_language.rfind("!")
  chosen_language, thread = chosen_language[0:position] , chosen_language[position+1::]

if not os.path.exists('tempaudios'):
  os.mkdir('tempaudios')
  audio_path="//content//tempaudios"

#if j[-5::]==".xlsx"] #show the excel tables that are in your current folder
#the ~$ exclude temporary files
tables_in_the_directory=[j for j in os.listdir() if j[-5::]==".xlsx" and j[0:2]!="~$"]

if len(tables_in_the_directory)>1:  #multiple tables to select
  while True:
    archive_name=input(translator.translate("Name of the excel table: ", src="en", dest=chosen_language).text)+".xlsx"
    if archive_name in tables_in_the_directory: break
    else:
      wrong_name=translator.translate("Wrong name!",src="en",dest=chosen_language)
      print(Back.RED + wrong_name.text,Style.RESET_ALL,end="")
else: archive_name=tables_in_the_directory[0]
path = os.path.join(os.getcwd(), archive_name)
archive ,deck_name= pd.read_excel(path), archive_name


##creating multiple decks with the same name
u,counter="",0
while os.path.exists(os.getcwd()+"\\"+deck_name+u+".apkg"):
    counter+=1
    u=str(counter)
deck_name+=str(u)

archive = archive.dropna()
archive.reset_index(drop=True, inplace=True)
dic_cardtype={"front":"vocabulary","speaking":"speaking","writing":"writing","pronunciation":"pronunciation"}
cardtype=dic_cardtype[archive.columns[0].lower()] #select the cardtype, doesn't matter if its lower or uppercase

if cardtype=="vocabulary":
  phrases = archive[[archive.columns[0]]]
  words = archive[[archive.columns[1]]]
elif cardtype=="speaking":
  phrases=archive[[archive.columns[0]]]
  print(cardtype)
  print(phrases)
elif cardtype=="writing":
  phrases=archive[[archive.columns[0]]]
elif cardtype=="pronunciation":
  pronunciationwords=archive[["Pronunciation"]]
  cardtype="pronunciation"
else:
    print(Back.RED + "ERROR")
    print(Style.RESET_ALL)
    front_message=translator.translate("Your excel table don't match with any type of table, this is probably due to bad formatting", src="en", dest=chosen_language)
    print(front_message.text)
if "Languages" in archive.columns:
    languages=[archive[["Languages"]].iloc[j][0] for j in range(len(archive))]
else:
    languages=[]

#Check if the words are in the phrase
j=0
## This function check if a exact word is in the phrase. it will not return true if for example
## phrase= "I worked a lot today"  word=work. This is different from the build-in function in python
## work is in "I worked a lot today", it'll return True, because "work" is in "worked"

if cardtype!="pronunciation":
    check_message=translator.translate("Do you want to check the cards before they are saved? (Yes = y No= n)",src="en",dest=chosen_language)
    check=input(check_message.text+": ")
else:
    check=""

def wordinphrase(word,phrase):
    word,phrase=str(word).lower(),str(phrase).lower()
    remove=[".",",","?","!",":",";"]
    for k in remove:
        phrase=phrase.replace(k," ")
    equal=0
    eachword=phrase.split()
    for j in eachword:
        if str(j)==str(word):
            equal+=1
    if equal==0:return False
    else: return True

if cardtype=='vocabulary':
    checkwordinphrase = input(translator.translate("Do you want to check if the word is in the phrase? (yes=y, no=n)", src="en",dest=chosen_language).text)
    if checkwordinphrase in ["Y","y","YES","yes","Yes"]:
        while j<len(phrases):
            if words.iloc[j][0].lower().replace(".","").replace(",","").replace(";","").replace(":","").replace("!",".").replace("?",".") not in phrases.iloc[j][0].lower().replace(".","").replace(",","").replace(";","").replace(":","").replace("!",".").replace("?",".") :
                word_notinphrase=translator.translate(f"Erro: Word not in the phrase ",src="en",dest=chosen_language)
                print(Back.RED + word_notinphrase.text,end="")
                print(Style.RESET_ALL)
                print(phrases.iloc[j][0])
                print(words.iloc[j][0])
                correctword_message = translator.translate("Correct word: ", src="en", dest=chosen_language)
                correctword=input(correctword_message.text)
                words.iloc[j][0] = correctword
                if wordinphrase(words.iloc[j][0].lower(),phrases.iloc[j][0].lower())==True:
                    j+=1
            else:j+=1
if len(languages)==0:
    unique_language_message=translator.translate("If there's just one unique language, type it here (Example: French=fr), if not, press any key: ",sr="en",dest=chosen_language)
    unique_language=input(unique_language_message.text)
    if len(unique_language)==2 and len(languages)==0: # if this is a language like en,fr,pt
        if cardtype!="pronunciation":
            for k in range(len(phrases)):
                languages.append(unique_language)
        else:
            for k in range(len(pronunciationwords)):
                languages.append(unique_language)
try:
    alpha=languages[0]
    alpha=1 #identify if there's a unique language or not
except:
    languages=[]
    alpha=0
translation_phrases = []
translation_words=[]
temp1=[]
temp2=[]
numbers=[]
print(translator.translate("Translating...(1/2)",src="en",dest=chosen_language).text)
if cardtype!="pronunciation":
    lenphrasestemp=len(phrases)

def translate(k,threads):
    global phrases,words,chosen_language,alpha,translation_words,translation_phrases,languages
    for j in range(k,len(phrases),threads):
        try:
            if alpha==0:
                    languages.append((translator.detect(phrases.iloc[j][0]).lang))
            t1=str(j)+" "+(translator.translate(phrases.iloc[j][0], src=str(languages[j]), dest=chosen_language)).text
            translation_phrases.append(t1)
            temp1.append(str(j)+" "+phrases.iloc[j][0])
            numbers.append(str(j))
            if cardtype=="vocabulary":
                t2=str(j)+" "+ (translator.translate(words.iloc[j][0], src=str(languages[j]), dest=chosen_language)).text
                translation_words.append(t2)
                temp2.append(str(j)+" "+words.iloc[j][0])
        except:pass

if cardtype!="pronunciation":
    processes = []
    for n in range(threads):
        p=Thread(target=translate,args=[n,threads])
        p.start()
        processes.append(p)
    for process in processes:
        process.join()
    translation_phrases=sorted(translation_phrases)
    phrases=sorted(temp1)
    numbers=sorted(numbers)
    if cardtype=="vocabulary":
        translation_words=sorted(translation_words)
        words=sorted(temp2)
    for j in range(len(phrases)):
        try:
            translation_phrases[j]=((translation_phrases[j]).replace(numbers[j],""))
            phrases[j]=(phrases[j].replace(numbers[j],""))
            if cardtype=="vocabulary":
                translation_words[j]=((translation_words[j]).replace(numbers[j],""))
                words[j]=(words[j].replace(numbers[j],""))
        except:pass
j=0


if check.lower()=="y" and cardtype=="vocabulary":
    if lenphrasestemp != len(phrases):
        print(Back.RED + translator.translate(
            f"If this numbers {lenphrasestemp} and {len(phrases)} are too different...So " + "You're using too much cores, that caused an error in the multiprocessing" + "\n" + "You can change the number of cores restarting the code" + "\n"
            + "In the chosen languages put the number of cores after ! (Ex: es !2)" + "\n" +
            "This way you are using 2 cores, in order to not cause erros put number of cores=1", src="en",
            dest=chosen_language).text + Style.RESET_ALL)
    checkphrases_message=translator.translate("Para mudar uma tradução escreva a nova tradução caso contrario deixe vazio"+"\n"+
                                              "É possível deletar um card, deletar=d"+ "\n"
                                              + "É possível voltar, voltar=b" + "\n" + "Se você quiser ir em uma frase especifica escreva j=numéro frase" +"\n"
                                              "Se o idioma estiver errado, digite !l no final da tradução nova",src="pt",dest=chosen_language)
    print(checkphrases_message.text)
    while j<len(phrases):
        try:
            print("----", j+1,"/",len(phrases),"-----")
            print(translator.translate("Language of the card",src="en",dest=chosen_language).text,Fore.LIGHTYELLOW_EX+ languages[j],Style.RESET_ALL)
            phrasehl=phrases[j].split()
            stringtoprint=""
            for k in range(len(phrasehl)):
                if phrasehl[k]==words[j].replace(" ",""):
                    position=k
                    stringtoprint+=Back.GREEN+phrasehl[k]+Style.RESET_ALL+ " "
                else: stringtoprint+=phrasehl[k] +" "
            print(stringtoprint)
            print("---"+translator.translate("Translation",src="en",dest=chosen_language).text+"---")
            translation=(translator.translate(phrases[j], src=str(languages[j]), dest=chosen_language)).text
            print(translation)
            print("---" + translator.translate("Translated word", src="en", dest=chosen_language).text + "---")
            print(Back.RED + words[j],"=",translation_words[j],end="")
            print(Style.RESET_ALL)
            newtranslation=input(("---"+translator.translate("New translation",src="en",dest=chosen_language).text)+": ")
            if newtranslation=="s":
                savefile = xlsxwriter.Workbook(deck_name+"stopped"+".xlsx")
                stoppedtable = savefile.add_worksheet()
                stoppedtable.write("A1", "Front")
                stoppedtable.write("B1", "Back")
                stop = j
                for k in range((len(phrases)-j)):
                    try:
                        stoppedtable.write("A"+str(j+k),phrases[j+k])
                        stoppedtable.write("B"+str(j+k),words[j+k])
                    except:
                        print(translator.translate("J to large",src="en",dest=chosen_language))
                j=len(phrases)+2
                savefile.close()
                break
            if "j=" in newtranslation:
                if j>len(phrases):
                    j=int(newtranslation[2::])
                    j=j-2
                else:
                    print(translator.translate("J to big, greater than the number of phrases",src="en",dest=chosen_language).text)
            if newtranslation.lower().count("!l")!=0:
                languages[j]=input(translator.translate("New language",src="en",dest=chosen_language).text+": ")
                newtranslation=newtranslation.replace("!l","")
            if len(newtranslation)!=newtranslation.count(" ") and newtranslation!="del" :
                if newtranslation.lower()!='b':
                    translation_words[j]=newtranslation
            if newtranslation.lower()=="d":
                del phrases[j], words[j],translation_words[j],translation_phrases[j]
            if newtranslation!="d": j+=1
            if newtranslation=="b":
                j=j-2
                try:newtranslation=translation_phrases[j+2]
                except:newtranslation=translation_phrases[j]
        except:
            del phrases[j]
            j+=1
if check.lower()=="y" and cardtype in ["speaking","writing"]:
    checkfile = xlsxwriter.Workbook(deck_name + "_check" + ".xlsx")
    checktable = checkfile.add_worksheet()
    checktable.write("A1", "speakingorwriting")
    checktable.write("B1", "Translation")
    checktable.write("C1", "Languages")
    for j in range(len(phrases)):
        checktable.write("A"+str(j+2),phrases[j])
        checktable.write("B"+str(j+2),translation_phrases[j])
        checktable.write("C"+str(j+2),languages[j])
    checkfile.close()
    input(translator.translate(f"Open the excel table and check if there's any mistake, press any key to continue",src="en",dest=chosen_language).text)
    path = os.path.join(os.getcwd(), deck_name+"_check" + ".xlsx")
    excelcheck = pd.read_excel(path)
    excelcheck = excelcheck.dropna()
    excelcheck.reset_index(drop=True, inplace=True)
    phrases = excelcheck[["speakingorwriting"]]
    translation_phrases = excelcheck[["Translation"]]
    languages=excelcheck[["Languages"]]
    templist1=[]
    templist2=[]
    templist3=[]
    for k in range(len(translation_phrases)):
        templist1.append(translation_phrases.iloc[k][0])
        templist2.append(phrases.iloc[k][0])
        templist3.append(languages.iloc[k][0])
    translation_phrases=templist1
    phrases=templist2
    languages=templist3
print(translator.translate("Audios...(2/2)",src="en",dest=chosen_language).text)
if cardtype!="pronunciation":stop = len(phrases)
else: stop=len(pronunciationwords)
def audiogenerator(k,threads):
    if cardtype!="pronunciation": global words,phrases,languages,audio_path,deck_name,stop
    else: global pronunciationwords,languages,audio_path,deck_name,stop
    for j in range(k, stop,threads):
        try:
            if cardtype == "vocabulary":
                tts = gTTS(str(words[j])+"."+str(phrases[j]), lang=languages[j])
                tts.save(audio_path + "//" + str(deck_name) + "phrase" + str(j) + '.mp3')
                tts = gTTS(str(words[j]), lang=languages[j])
                tts.save(audio_path + "//" + str(deck_name) + "word" + str(j) + '.mp3')
            if cardtype in ["speaking","writing"]:
                tts = gTTS(str(phrases[j]), lang=languages[j])
                tts.save(audio_path + "//" + str(deck_name) + "phrase" + str(j) + '.mp3')
            if cardtype=="pronunciation":
                tts = gTTS(str(pronunciationwords.iloc[j][0]), lang=languages[j])
                tts.save(audio_path + "//" + str(deck_name) + "pronunciation" + str(j) + '.mp3')
        except:pass

processes=[]
for n in range(threads):
    p=Thread(target=audiogenerator,args=[n,threads])
    p.start()
    processes.append(p)
for process in processes:
    process.join()
if cardtype=="vocabulary" or "speaking":
    id_deck =1_335_132_555
if cardtype=="writing":
    id_deck=2_343_103_533
if cardtype=="pronunciation":
    id_deck=3_304_103_941
deck = genanki.Deck(
    id_deck,
    deck_name)
if cardtype in ["vocabulary","speaking"]:
    my_model = genanki.Model(
        id_deck,
        'CardMaker Q&A',
        fields=[
            {'name': 'Question'},
            {'name': 'Answer'},
            {'name': 'MyMedia'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Question}}<br>{{MyMedia}}',
                'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
            },
        ])
if cardtype=="writing":
    my_model = genanki.Model(
        id_deck,
        'CardMaker Type in the Answer',
        fields=[
            {'name': 'Question'},
            {'name': 'Answer'},
            {'name': 'MyMedia'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Question}}<br> {{type:Answer}}',
                'afmt': '{{FrontSide}}{{MyMedia}}',
            },
        ])
if cardtype=="pronunciation":
    my_model = genanki.Model(
        id_deck,
        'CardMaker Pronunciation',
        fields=[
            {'name': 'Question'},
            {'name': 'Answer'},
            {'name': 'MyMedia'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Question}}<br>{{MyMedia}}',
                'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
            },
        ])
#Color of the cards

print(translator.translate("Colors that are in the code: ",src="en",dest=chosen_language).text)
print("blue,green,red,purple,pink,yellow")
custom_color_message=translator.translate("You can select the color of the highlighted word, if you want to create a new color press yes=y."+ "\n"+"If you want a predetermined color write the name of the color",src="en",dest=chosen_language)
color=input(custom_color_message.text+": ")
colors_rgb = {'green': '<span style="color: rgb(81, 255, 37);">', 'red': '<span style="color: rgb(228, 14, 14);">',
              'blue': '<span style="color: rgb(18, 166, 252);">',
              'yellow': '<span style="color: rgb(249, 255, 54);">',
              'purple': '<span style="color: rgb(198, 38, 255);">',
              'pink': '<span style="color: rgb(255, 14, 192);">', }

if color in ["Y","y","YES","yes","Yes"]:
    try:
        driver = webdriver.Chrome()
        driver.get("https://www.rapidtables.com/web/color/RGB_Color.html")
    except: print("https://www.rapidtables.com/web/color/RGB_Color.html")
    R=input("Red: ")
    G=input("Green: ")
    B=input("Blue: ")
    driver.close()
    color=f'<span style="color: rgb({R}, {G}, {B});">'
else:
    try:
        color=colors_rgb[color]
    except:
        color=colors_rgb["red"]
for i in range(0, stop):
    try:
        if cardtype=="speaking":
            note = genanki.Note(model=my_model,
                                fields=[color + '<u><b><i>' +" ["+ languages[i]+"] "+'</i></b></u></span>' +
                                        translation_phrases[i],"[" + "sound:" + str(deck_name) + "phrase" + str(i) + ".mp3" + "]" + str(" " + phrases[i])
                                    ,"" ],tags=[str(languages[i]), "cardmaker"])
        if cardtype=="vocabulary":
            note = genanki.Note(model=my_model,
                    fields=["","[" + "sound:" + str(deck_name) + "word" + str(i) + ".mp3" + "]" + color + '<u><b><i>' + str(" "+ words[i]) + '</i></b></u></span>'  " == " + str(
                                translation_words[i]), "[" + "sound:" + str(deck_name) + "phrase" + str(i) + ".mp3" + "]"  + color+ '<u><b><i>' +" "+words[i]+ '</i></b></u></span>' + ". " +
                        phrases[i]],tags=[str(languages[i]),"cardmaker"])
        if cardtype=="writing":
            note = genanki.Note(model=my_model,
                                fields=[color + '<u><b><i>' + " [" + languages[i] + "] " + '</i></b></u></span>' +
                                        translation_phrases[i], phrases[i],"[" + "sound:" + str(deck_name) + "phrase" + str(i) + ".mp3" + "]"], tags=[str(languages[i]), "cardmaker"])
        if cardtype=="pronunciation":
            note = genanki.Note(model=my_model,
                                fields=[color+ '<u><b><i>' + pronunciationwords.iloc[i][0] + '</i></b></u></span>' ,
                                        "[" + "sound:" + str(deck_name) + "pronunciation" + str(i) + ".mp3" + "] "+
                                    pronunciationhelp[i], ""], tags=[str(languages[i]), "cardmaker"])
        deck.add_note(note)
    except:pass


genanki.Package(deck).write_to_file(str(deck_name) +'.apkg')


readme=open("readme.txt","w")
readme.write("copy this audios into your collection.media anki folder")
readme.close()

with ZipFile("audios.zip","w") as zip:
  zip.write("readme.txt")
  for j in os.listdir("tempaudios/"):
    zip.write("tempaudios"+"//"+j)
    os.remove("//content//tempaudios"+"//"+j)
os.remove("readme.txt")
os.rmdir('tempaudios')

tf=time.time()

deltat=tf -t_o
if cardtype!="pronunciation":
    print(f"Congratulations, {len(phrases)} flashcards in {round(deltat/60,1)} minutes! {round(60*len(phrases)/deltat,1)} flashcards per minute")
else:
    print(f"Congratulations, {len(pronunciationwords)} flashcards in {round(deltat / 60, 1)} minutes! {round(60 * len(pronunciationwords) / deltat, 1)} flashcards per minute")


Choose the language that your cards and the interface will be translated (ex:pt,fr,en): pt
speaking
                                            Speaking
0  ¿Qué resultados pretendes encontrar con esta i...
1  La historia era mi asignatura favorita en el c...
2  Estoy en mi cuarto curso en la facultad de med...
3  La Facultad de Ingeniería está en el centro de...
4  El Ministerio de Educación ha aprobado un nuev...
5                 ¿Cuál de estas es la mochila tuya?
6                    ¡Esta película es tan aburrida!


KeyboardInterrupt: ignored